In [1]:
#################### Includes modules
import numpy as np
import glob as gl
import csv
import pandas
import h5py
import os
import import_ipynb
from decimal import Decimal, getcontext
#import Dataset as dtst
import NeuralNetwork as nn

importing Jupyter notebook from NeuralNetwork.ipynb


In [4]:
#################### Main program
# Classification types: 1 -> for loading files with data from the scope of valve opening range = 5%
#                       2 -> for files with samples for every 1% of valve opening range
#                       3 -> for loading all files
#                       4 -> for loading data from files with valve opening range every 10%

dt = dtst.Dataset()
print("===========1")
print(dt.validation_valve_opening_scopes)
print("===========2")
print(dt.training_valve_opening_scopes)
print("===========")
np.set_printoptions(threshold=sys.maxsize)
print(dt.validation_data)
#print(dt.training_data)
print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
model = nn.ConvClass()
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'], shuffle=True)
model.fit(dt.training_data, dt.training_valve_opening_scopes, epochs=100)
_, accuracy = model.evaluate(dt.training_data, dt.training_valve_opening_scopes, verbose=0)
model.summary()


NameError: name 'dtst' is not defined

In [ ]:
# make class predictions with the model
datatopredict= np.expand_dims(dt.validation_data, axis=2)
predictions = model.predict_classes(datatopredict)
predictiontestlist = []
predictionfullinfo = []

for i in range(predictions.__len__()):
  #print('%s => %d (expected )' % (datatopredict[i].tolist(), predictions[i]))
  s = 'Data sample numer: %d \nResult for data set: %s => %d (expected %d)' % (i, datatopredict[i].tolist(), predictions[i], dt.validation_valve_opening_scopes[i])
  predictionfullinfo.append(s)
  if predictions[i] == dt.validation_valve_opening_scopes[i]:
    result = 1
  else:
    result = 0
  predictiontestlist.append(result)

wrongresultepochs = predictiontestlist.count(0)
goodresultepochs = predictiontestlist.count(1)

print("Wrong/Good predictions: %d / %d" % (wrongresultepochs , goodresultepochs))
print("Predictions details: ")
for i in range(predictions.__len__()):
  print(predictionfullinfo[i])

In [3]:
### Nowy start z testowaniem dla CONV2D z tradycyjna klasyfikacja co 5% otwarcia zaworu
FILES_WITH_DATA = gl.glob("../Data/*.mat")
CONV2D_AXIS_X = 43
CONV2D_AXIS_Y = 7
SAPLES_NUMBER = 20
DATASET = np.zeros([SAPLES_NUMBER * 20, CONV2D_AXIS_X, CONV2D_AXIS_Y])
DATA_LABELS = np.zeros([SAPLES_NUMBER * 20])

last_valve_value = 0
sn_iter = 0
x_iter = 0
y_iter = 0
file_iter = 0
for f in FILES_WITH_DATA:
    valve_value = float(os.path.basename(f)[:-4])
    if (valve_value // 1 % 5 == 0):
        if ((last_valve_value // 1) != (valve_value // 1)):
            last_valve_value = valve_value
            tmp_array_of_full_data_from_file = np.array(h5py.File(f).get('p').value)
            for sn in range(sn_iter, sn_iter + SAPLES_NUMBER):
                for x in range(CONV2D_AXIS_X):
                    for y in range(CONV2D_AXIS_Y):
                        DATASET[sn][x][y] = tmp_array_of_full_data_from_file[file_iter][y]
                    file_iter += 1
                DATA_LABELS[sn] = int(last_valve_value // 1)
            sn_iter += SAPLES_NUMBER

print(DATASET[0])

C:\Users\Kowalio\Miniconda3\envs\tensorflow\lib\site-packages\h5py\_hl\dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)


[[ 6.96887999e+00 -9.52249656e-02  4.77578900e+00  1.52860919e+00
   1.14492709e+00 -8.42782085e-02 -2.37996143e-02]
 [ 6.88547025e+00 -1.60369667e-01  4.93528326e+00  1.48725764e+00
   1.07296800e+00 -1.52814848e-01 -1.01507233e-01]
 [ 6.73381618e+00 -3.70280371e-01  4.53123114e+00  1.32930828e+00
   9.85744852e-01 -4.38384177e-01 -3.03930785e-01]
 [ 6.88736593e+00 -7.27862351e-02  4.75186486e+00  1.49539238e+00
   1.06497254e+00 -1.06171857e-01 -8.90356403e-02]
 [ 6.87220052e+00 -1.63264987e-01  4.81034609e+00  1.39235245e+00
   1.06860684e+00 -1.97554042e-01 -8.04014604e-02]
 [ 6.76983402e+00 -2.55915229e-01  4.58439590e+00  1.37811667e+00
   1.08314403e+00 -2.96551410e-01 -1.67702613e-01]
 [ 6.82196511e+00 -2.87039919e-01  4.70401659e+00  1.40997770e+00
   1.04462047e+00 -3.29867832e-01 -1.77296146e-01]
 [ 6.91959242e+00 -1.45893067e-01  4.95566308e+00  1.50149342e+00
   1.06715312e+00 -1.67093314e-01 -9.95885268e-02]
 [ 6.87599187e+00  6.18461476e-02  4.98933409e+00  1.66012068e+0

In [7]:
from tensorflow.keras.utils import to_categorical

In [9]:
DATASET = DATASET.reshape(SAPLES_NUMBER * 20, CONV2D_AXIS_X, CONV2D_AXIS_Y,1)   
print(DATA_LABELS)
DATA_LABELS = to_categorical(DATA_LABELS)
model = nn.ConvClass()
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'], shuffle=True)
model.fit(DATASET, DATA_LABELS, epochs=100)
_, accuracy = model.evaluate(DATASET, DATA_LABELS, verbose=0)
print(accuracy)
model.summary()

[ 10.  10.  10.  10.  10.  10.  10.  10.  10.  10.  10.  10.  10.  10.
  10.  10.  10.  10.  10.  10. 100. 100. 100. 100. 100. 100. 100. 100.
 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100. 100.  15.  15.
  15.  15.  15.  15.  15.  15.  15.  15.  15.  15.  15.  15.  15.  15.
  15.  15.  15.  15.  20.  20.  20.  20.  20.  20.  20.  20.  20.  20.
  20.  20.  20.  20.  20.  20.  20.  20.  20.  20.  25.  25.  25.  25.
  25.  25.  25.  25.  25.  25.  25.  25.  25.  25.  25.  25.  25.  25.
  25.  25.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.  30.
  30.  30.  30.  30.  30.  30.  30.  30.  35.  35.  35.  35.  35.  35.
  35.  35.  35.  35.  35.  35.  35.  35.  35.  35.  35.  35.  35.  35.
  40.  40.  40.  40.  40.  40.  40.  40.  40.  40.  40.  40.  40.  40.
  40.  40.  40.  40.  40.  40.  45.  45.  45.  45.  45.  45.  45.  45.
  45.  45.  45.  45.  45.  45.  45.  45.  45.  45.  45.  45.   5.   5.
   5.   5.   5.   5.   5.   5.   5.   5.   5.   5.   5.   5.   5.   5.
   5. 

ValueError: Dimensions must be equal, but are 20 and 101 for 'loss/output_1_loss/SquaredDifference' (op: 'SquaredDifference') with input shapes: [?,20], [?,101].